### Шаг 1. Открытие файла с данными и изучение общей информации. 

In [2]:
import pandas as pd
data = pd.read_csv('/home/anna/Рабочий стол/Практикум/данные для проектов/data.csv')
print(data)

data.info()

print(data['total_income'].head(20))


       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1   -4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1   -2113.346888         38   среднее             1   
21523         3   -3112.481705         38   среднее             1   
21524         2   -1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

### Вывод

Изучив файл, делаю вывод, что в нем 21525 строк, 12 столбцов. В двух столбцах дробные числа, 5 столбцов с целыми числами и еще 5 столбцов с типом данных object. В столбцах days_employed и total_income имеются пропущенные значения.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#с помощью метода isnull() посмотрим в каких столбцах есть нулевые значения и сколько их
print(data[data['total_income'].isnull()].count())

#считаем среднюю по столбцу с ежемесячным доходом, чтобы убрать пропуски и заменить на среднее
total_income_mean = data['total_income'].mean()

print(total_income_mean)

#с помощью метода fillna() удаляю пропуски и заменяю на среднее
data['total_income'] =data['total_income'].fillna(value=total_income_mean)

#проверка
data.info()
print(data['total_income'].head(15))




children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64
167422.30220817294
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int

Вывод

В 2174 строках из 21525 значения общего трудового стажа и ежемесячного дохода пропущены одновременно. Возможно люди, у которых есть пропуски, совсем недавно устроились на работу (в течении последнего месяца), поэтому по ежемесячному доходу и стажу данных пока нет.  Всего пропущенных значений 10,1%, что довольно много и просто удалить строки не целесообразно. Кроме того, можно заметить ,что данные по трудовому стажу некорректны , а так как для итоговых целей данного проекта нам данные по трудовому стажу не нужны, то и преобразовывать их смысла нет. Для обработки пропусков в столбце "Ежемесячный доход" заменила пропущенные значения по доходу на средний доход, который составляет 167 тыс.


### Замена типа данных

In [4]:
#уберем шесть знаков после запятой в столбце со средним доходом:
data['total_income'] = data['total_income'].astype(int)
data.info()
print(data['total_income'].head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64


### Вывод

Я преобразовала данные по ежемесячному доходу.  Убрала ненужные  6 цифр после запятой, заменив вещественный тип данных(float) на целочисленный(int) с помощью метода astype. Использовала этот метод, потому что  у нас изначально значения числовые - float, поэтому нам не нужен метод to_numeric(). Проверила вызвав info, а также напечатав первые 5 строк измененного столбца.

### Обработка дубликатов

In [5]:
#смотрю наличие дубликатов
print(data.duplicated().value_counts())

#Выяснила, что есть 54 дубликата. Но в столбце "education" есть одинаковые слова, но с разным регистром. Приведу все слова 
#к нижнему регистру.

data['education'] = data['education'].str.lower()
print(data.duplicated().value_counts())

#количество дубликатов увеличилось с 54 до 71. Теперь удалю их.
data = data.drop_duplicates().reset_index(drop=True)

#проверим результат
print(data.duplicated().value_counts())

#проверка
print(data['education'].value_counts())



False    21471
True        54
dtype: int64
False    21454
True        71
dtype: int64
False    21454
dtype: int64
среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


### Вывод

С помощью метода duplicated  и value_counts выяснила, что всего повторяющихся строк-дубликатов - 54. В столбце образование привели все слова к нижнему регистру методом str.lower(). После этого число дубликатов увеличилось и стало 71. Удалила их методом drop_duplicates с изменением индексов и удалением старых (reset_index(drop=True)). Проверила результат dublicated() и value_counts().

### Лемматизация

In [6]:
#в лемматизации явно нуждается столбец с целями кредита, применю к нему метод value_counts()
print(data['purpose'].value_counts())

from pymystem3 import Mystem
m = Mystem()
#лемматизируем столбец с целями кредита
#Чтобы применять лемматизаию к каждой строке, создадим функцию
def lemmatize_data(row):
    lemmas = m.lemmatize(row)
    return lemmas

#воспользуемся методом apply, чтобы применить созданную функцию к столбцу с целями кредита
lemmas_purpose = data['purpose'].apply(lemmatize_data)
print(lemmas_purpose)



свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Я лемматизировала столбец с целями кредита, используя функцию и применив метод apply

### Категоризация данных

In [7]:
#для того, чтобы ответить на поставленные вопросы категоризируем данные по стобцам: children, family_status,  total_income 
#и purpose

#сгруппируем количество задолженностей по количеству детей методом value_counts
#print(data['children'].value_counts())

#есть ошибки в данных о детях. 20 - это скорее всего 2, а -1 это 1. Исправим ошибки методом replace
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)

#проверим результат
print(data['children'].value_counts())

#категоризируем данные по количеству детей:
def children_group(count):
    if count == 0:
        return 'бездетный'
    elif 1 <= count <= 2:
        return 'имеется 1 или 2 детей'
    else:
        return 'многодетный'
#проверка    
print(children_group(1))

#добавим столбец с созданной категоризацией по наличию детей
data['children_group'] = data['children'].apply(children_group)
print(data['children_group'].head(10))


#применим метод value_counts() к столбцу с семейным положением:
print(data['family_status'].value_counts())
#на мой взгляд столбец 'family_status' не нуждается в категоризации,у него и так корректная группировка

#Категоризируем столбец с целями кредита purpose:
for i in range(len(lemmas_purpose)):
    if 'жилье' in lemmas_purpose[i] or 'недвижимость' in lemmas_purpose[i]:
        lemmas_purpose[i] = 'недвижимость'
    elif 'образование' in lemmas_purpose[i]:
        lemmas_purpose[i] = 'образование'
    elif 'свадьба' in lemmas_purpose[i]:
        lemmas_purpose[i] = 'свадьба'
    else:
        lemmas_purpose[i] = 'автомобиль'
print(lemmas_purpose)

#добавлю созданный столбец lemmas_purpose к общей таблице:
data_new = data
data_new['purpose_group'] = lemmas_purpose
print(data_new.head(10))


#категоризируем столбец с ежемесячным доходом "total income". Для этого сначала расчитаем максимальное, минимальное и среднее значения:
print(data_new['total_income'].max())
print(data_new['total_income'].mean())
print(data_new['total_income'].min())

#разобьем на группы
def total_income_group(count):
    if count <= 50000:
        return 'низкий доход'
    elif 50001 <= count <= 300000:
        return 'средний доход'
    else:
        return 'высокий доход'
#проверка    
print(total_income_group(100000))

#добавим столбец с созданной категоризацией по ежемесячному доходу:
data['total_income_group'] = data['total_income'].apply(total_income_group)
print(data['total_income_group'].head(10))


0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64
имеется 1 или 2 детей
0    имеется 1 или 2 детей
1    имеется 1 или 2 детей
2                бездетный
3              многодетный
4                бездетный
5                бездетный
6                бездетный
7                бездетный
8    имеется 1 или 2 детей
9                бездетный
Name: children_group, dtype: object
женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64
0        недвижимость
1          автомобиль
2        недвижимость
3         образование
4             свадьба
             ...     
21449    недвижимость
21450      автомобиль
21451    недвижимость
21452      автомобиль
21453      автомобиль
Name: purpose, Length: 21454, dtype: object
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42

### Вывод

In [8]:
### Код ревьювера ###

display(data['total_income'].quantile([0.25, 0.5, 0.75]))
display(data['total_income'].median())

0.25    107623.00
0.50    156043.50
0.75    195813.25
Name: total_income, dtype: float64

156043.5

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
#сгруппируем все задолженности из столбца debt по столбцу с количеством детей
children_debt = data_new.groupby('children_group')['debt'].sum()
print(children_debt)

#с помощью метода value_counts() посчитаем количество человек по family группам
children_group_count = data_new['children_group'].value_counts()
print(children_group_count)

#найдем отношение сумм задолженностей к количеству клиентов по группам
children_debt_procent = children_debt / children_group_count

print('Доля задолженностей у бездетных клиентов: {:.2%}'.format(children_debt_procent[0]))
print('Доля задолженностей у клиентов с 1 и 2 детьми: {:.2%}'.format(children_debt_procent[1]))
print('Доля задолженностей у многодетных клиентов: {:.2%}'.format(children_debt_procent[2]))

children_group
бездетный                1063
имеется 1 или 2 детей     647
многодетный                31
Name: debt, dtype: int64
бездетный                14091
имеется 1 или 2 детей     6983
многодетный                380
Name: children_group, dtype: int64
Доля задолженностей у бездетных клиентов: 7.54%
Доля задолженностей у клиентов с 1 и 2 детьми: 9.27%
Доля задолженностей у многодетных клиентов: 8.16%


### Вывод

Мы видим, что наибольшее количество задолженностей по кредиту (9.27%) у клиентов, имеющих 1 или 2 детей. А наименьшее количество задолженностей у бездетных клиентов(7.54%). Хоть и процент отличается не сильно, но зависимость все же есть. В случае спорного решения по другим факторам, банку стоит отдать предпочтение бездетным клиентам.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
#сгруппируем все задолженности из столбца debt по столбцу с семейным статусом:
family_status_debt = data_new.groupby('family_status')['debt'].sum()
print(family_status_debt)

family_status_count = data_new['family_status'].value_counts()
print(family_status_count)

family_status_procent = family_status_debt / family_status_count

print('Доля задолженностей у клиентов со статусом - Не женат / не замужем : {:.2%}'.format(family_status_procent[0]))
print('Доля задолженностей у клиентов со статусом - в разводе {:.2%}'.format(family_status_procent[1]))
print('Доля задолженностей у клиентов со статусом - вдовец / вдова: {:.2%}'.format(family_status_procent[2]))
print('Доля задолженностей у клиентов со статусом - гражданский брак: {:.2%}'.format(family_status_procent[3]))
print('Доля задолженностей у клиентов со статусом - женат / замужем: {:.2%}'.format(family_status_procent[4]))

family_status
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
Name: debt, dtype: int64
женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64
Доля задолженностей у клиентов со статусом - Не женат / не замужем : 9.75%
Доля задолженностей у клиентов со статусом - в разводе 7.11%
Доля задолженностей у клиентов со статусом - вдовец / вдова: 6.57%
Доля задолженностей у клиентов со статусом - гражданский брак: 9.35%
Доля задолженностей у клиентов со статусом - женат / замужем: 7.55%


### Вывод

По данной выборке мы видим, что наибольший процент задолженности у тех клиентов, которые не находятся в официальном браке. Это можно объяснить тем, что люди не хотят брать ответственность, регистрируя официальный брак. Возможно в какой то степени у них снижена и ответственность по выплате кредита. Проценты варьируются не сильно (от 6 до 9%)в зависимости от семейного статуса, но все же банкам стоит отдавать предпочтение женатым/замужним клиентам, либо в прошлом находящимся в официальном браке (вдовцам/разведенным).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
#сгруппируем все задолженности из столбца debt по столбцу с целями кредита:
total_income_debt = data_new.groupby('total_income_group')['debt'].sum()
print(total_income_debt)

total_income_count = data_new['total_income_group'].value_counts()
print(total_income_count)

total_income_procent = total_income_debt / total_income_count

print('Доля задолженностей у клиентов с высоким доходом : {:.2%}'.format(total_income_procent[0]))
print('Доля задолженностей у клиентов с низким доходом {:.2%}'.format(total_income_procent[1]))
print('Доля задолженностей у клиентов со средним доходом: {:.2%}'.format(total_income_procent[2]))


total_income_group
высокий доход     106
низкий доход       23
средний доход    1612
Name: debt, dtype: int64
средний доход    19600
высокий доход     1482
низкий доход       372
Name: total_income_group, dtype: int64
Доля задолженностей у клиентов с высоким доходом : 7.15%
Доля задолженностей у клиентов с низким доходом 6.18%
Доля задолженностей у клиентов со средним доходом: 8.22%


### Вывод

Удивительно, но по данной выборке, люди с высоким доходом имеют чуть больше задолженностей, чем люди с низким доходом. Проценты варьируются не значительно, от 6 до 8%. Думаю эта выборка говорит о том, что возврат кредита в срок не зависит от уровня дохода. Потому что в любом случае банк выдает соразмерную доходу клиента сумму, поэтому вероятность выплаты кредита в срок зависит от ответственности клиента и прочих факторов, но не от суммы дохода. 

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
#сгруппируем все задолженности из столбца debt по столбцу с целями кредита:
purpose_debt = data_new.groupby('purpose_group')['debt'].sum()
print(purpose_debt)

purpose_count = data_new['purpose_group'].value_counts()
print(purpose_count)

purpose_procent = purpose_debt / purpose_count

print('Доля задолженностей у клиентов с целью кредита - автомобиль : {:.2%}'.format(purpose_procent[0]))
print('Доля задолженностей у клиентов с целью кредита - недвижимость {:.2%}'.format(purpose_procent[1]))
print('Доля задолженностей у клиентов с целью кредита - образование: {:.2%}'.format(purpose_procent[2]))
print('Доля задолженностей у клиентов с целью кредита - свадьба: {:.2%}'.format(purpose_procent[3]))

purpose_group
автомобиль      403
недвижимость    782
образование     370
свадьба         186
Name: debt, dtype: int64
недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64
Доля задолженностей у клиентов с целью кредита - автомобиль : 9.36%
Доля задолженностей у клиентов с целью кредита - недвижимость 7.23%
Доля задолженностей у клиентов с целью кредита - образование: 9.22%
Доля задолженностей у клиентов с целью кредита - свадьба: 8.00%


### Вывод

Данная выборка говорит о том, что люди, покупая недвижимость, с большей ответственностью относятся к кредиту, чем те, кто берет кредит на автомобиль или образование. Тому есть логическое объяснение. Покупка/строительство недвижимости требует более взвешенного решения и сумма там внушительнее. Когда как автомобиль можно купить иногда спонтанно. Кредит на образование часто берут люди, которые планируют улучшить свое финансовое положение с помощью получения образования, планы у них могут меняться, текущее финансовое положение тоже, поэтому данная цель тоже с довольно высоким процентом задолженности (9,22%). Кредит на свадьбу занимает среднее положение (8%), зачастую молодожены рассчитывают вернуть кредит с денег, подаренных на свадьбу, но не всегда ожидания оправдываются.

### Шаг 4. Общий вывод

Судя по представленным данным, идеальный клиент с наибольшей вероятностью выплаты кредита в срок  - это бездетный женатый/замужний человек, берущий кредит на недвижимость. Либо бездетный вдовец/разведенный. Это вполне логично, так как дети являются фактором риска, ведь в некоторых случаях на них можно потратить все имеющиеся средства и получить просрочку по кредиту.  А люди, берущие на себя ответственность, официально вступая в семейные отношения, вероятно и более ответственно подходят к ответственности перед банком. 

Клиент с наименьшей вероятностью выплаты кредита в срок - неженатый/незамужний клиент с 1 или 2 детьми, решивший взять кредит на автомобиль или образование. Что тоже логично.